<a href="https://colab.research.google.com/github/gabbyprecious/hng-conversationalAnalysisUsingGPT2/blob/master/conversational_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#importing libraries
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [0]:
#cloning git
!git clone https://github.com/tenoke/gpt-2

Cloning into 'gpt-2'...
remote: Enumerating objects: 244, done.
remote: Total 244 (delta 0), reused 0 (delta 0), pack-reused 244
Receiving objects: 100% (244/244), 4.39 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [0]:
cd gpt-2

/content/gpt-2


In [0]:
#installing necessary requirements
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 604kB 14.8MB/s 
     |████████████████████████████████| 51kB 14.8MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=179b8f205c8721d956ba023c5b73040ed2af84620be38a40efc4405a47f0b470
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533168 sha256=d2fec77596b4f6bbc0a9a36de4cfe7a830e478d211acfc092ca7c54a023c4a1c
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"basifrank","key":"f5f3c7a9c1f89b369d085833c612e5f3"}'}

In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Oct 12 09:02 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json
#downloading data
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus

100% 798M/799M [00:11<00:00, 70.3MB/s]
100% 799M/799M [00:11<00:00, 75.7MB/s]


 99% 789M/799M [00:09<00:00, 116MB/s]
100% 799M/799M [00:09<00:00, 87.7MB/s]


In [0]:
!unzip ubuntu-dialogue-corpus.zip



Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


In [0]:
#Reading data
FIRST_DATA = 'Ubuntu-dialogue-corpus/dialogueText.csv'
SECOND_DATA = 'Ubuntu-dialogue-corpus/dialogueText_196.csv'
THIRD_DATA = 'Ubuntu-dialogue-corpus/dialogueText_301.csv'

DF1 = pd.read_csv(FIRST_DATA, parse_dates=['date'], chunksize=1200000)
DF2 = pd.read_csv(SECOND_DATA, parse_dates=['date'], chunksize=1200000)
DF3 = pd.read_csv(THIRD_DATA, parse_dates=['date'], chunksize=1200000)

In [0]:
#downloading model
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 587kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 36.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 582kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:23, 61.7Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.46Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 59.0Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 48.1Mit/s]                                                       


In [0]:
!mkdir ubuntu-data ubuntu-npz

In [0]:
#Cleaning function
def clean(dataset, file_name):
  i = 1
  for data in dataset:    
    text_corpus = ''
    current = None
    for msg in data.itertuples():
      if msg.dialogueID != current:
        current = msg.dialogueID
        text_corpus += '\n\n'
      try: 
        text_corpus += f"({msg.date}) {msg._4}: {msg.text}\n"
      except KeyError:
        pass
    
    
    with open(f'ubuntu-data/{file_name}_{i}.txt', 'w') as f:
      f.write(text_corpus)  
    del(text_corpus)
    i += 1

In [0]:
#cleaning data
clean(DF1, 'cleaned_d1_df')
clean(DF2, 'cleaned_d2_df')
clean(DF3, 'cleaned_d3_df')

In [0]:
#encoding
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d1_df_1.txt ubuntu-npz/cleaned_d1_df_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_1.txt ubuntu-npz/cleaned_d2_df_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_2.txt ubuntu-npz/cleaned_d2_df_2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_3.txt ubuntu-npz/cleaned_d2_df_3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_4.txt ubuntu-npz/cleaned_d2_df_4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_5.txt ubuntu-npz/cleaned_d2_df_5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_6.txt ubuntu-npz/cleaned_d2_df_6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_7.txt ubuntu-npz/cleaned_d2_df_7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d2_df_8.txt ubuntu-npz/cleaned_d2_df_8.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_1.txt ubuntu-npz/cleaned_d3_df_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_2.txt ubuntu-npz/cleaned_d3_df_2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_3.txt ubuntu-npz/cleaned_d3_df_3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_4.txt ubuntu-npz/cleaned_d3_df_4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_5.txt ubuntu-npz/cleaned_d3_df_5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_6.txt ubuntu-npz/cleaned_d3_df_6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_7.txt ubuntu-npz/cleaned_d3_df_7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_8.txt ubuntu-npz/cleaned_d3_df_8.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_9.txt ubuntu-npz/cleaned_d3_df_9.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_d3_df_10.txt ubuntu-npz/cleaned_d3_df_10.txt.npz --model_name 345M

Reading files
100% 1/1 [04:27<00:00, 267.28s/it]
Writing ubuntu-npz/cleaned_d1_df_1.txt.npz
Reading files
100% 1/1 [03:50<00:00, 230.74s/it]
Writing ubuntu-npz/cleaned_d2_df_1.txt.npz
Reading files
100% 1/1 [03:59<00:00, 239.39s/it]
Writing ubuntu-npz/cleaned_d2_df_2.txt.npz
Reading files
100% 1/1 [03:49<00:00, 229.92s/it]
Writing ubuntu-npz/cleaned_d2_df_3.txt.npz
Reading files
100% 1/1 [03:46<00:00, 226.81s/it]
Writing ubuntu-npz/cleaned_d2_df_4.txt.npz
Reading files
100% 1/1 [03:54<00:00, 234.08s/it]
Writing ubuntu-npz/cleaned_d2_df_5.txt.npz
Reading files
100% 1/1 [04:11<00:00, 251.50s/it]
Writing ubuntu-npz/cleaned_d2_df_6.txt.npz
Reading files
100% 1/1 [04:09<00:00, 249.44s/it]
Writing ubuntu-npz/cleaned_d2_df_7.txt.npz
Reading files
100% 1/1 [02:29<00:00, 149.05s/it]
Writing ubuntu-npz/cleaned_d2_df_8.txt.npz
Reading files
100% 1/1 [03:53<00:00, 233.36s/it]
Writing ubuntu-npz/cleaned_d3_df_1.txt.npz
Reading files
100% 1/1 [04:06<00:00, 246.09s/it]
Writing ubuntu-npz/cleaned_d3_d

In [0]:
#Training data
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --stop_after 1450 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 10:44:17.057088: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 10:44:17.061403: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x24ed640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 10:44:17.061499: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [0]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --learning_rate 0.0001 --stop_after 3501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 10:50:33.265470: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 10:50:33.265830: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1bb7640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 10:50:33.265880: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [0]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --learning_rate 0.000001 --stop_after 5001 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 10:56:15.299253: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 10:56:15.299625: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1c2f640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 10:56:15.299673: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [0]:
#Training model

!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every=250 --learning_rate=0.01 --stop_after=1251 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 11:03:17.991864: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 11:03:17.992235: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1f65640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 11:03:17.992278: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [0]:
#copy the new trainned weights


!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

cp: cannot stat '/content/gpt-2/checkpoint/run1/*': No such file or directory


In [0]:
#generate conversations on its own 
#generate sample articles on its own base on the trained data.
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.95 --model_name 345M



2019-10-12 11:14:47.496973: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 11:14:47.497253: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x27ad800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 11:14:47.497290: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

======================================== SAMPLE 1 ========================================
With the rise of Apple Pay, it's easy to ignore how much less data Apple charges. Some consumers may still think that, but it would be more than the $99 fee—or just the $20.99 charge, if you're a customer of AT&T who uses AT&T's network.

The company's new mobi

In [0]:
#generate article on its own after finetuning model temperature and top_k
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 1 --model_name 345M



2019-10-12 11:37:46.500251: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 11:37:46.500513: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x15bf800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 11:37:46.500548: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

======================================== SAMPLE 1 ========================================
Video

In February this year, a group of French scientists took measurements of a mysterious white substance that is found on a meteorite and which can be described from space as a "white cloud".

Some experts say if the substance were not an asteroid or comet 

In [0]:
!python3 src/interactive_conditional_samples.py  --model_name='345M'  --nsamples=20 --top_k=40 --temperature=.90 --length=1



2019-10-12 11:51:16.412958: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 11:51:16.413238: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x18ff800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 11:51:16.413276: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

Model prompt >>> 137378602389
======================================== SAMPLE 1 ========================================
37
======================================== SAMPLE 2 ========================================
7
======================================== SAMPLE 3 ========================================
77
========================================

In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M' --nsamples=2 --top_k=40 --temperature=.90



2019-10-12 11:54:48.899219: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 11:54:48.899461: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1cc3800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 11:54:48.899493: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

Model prompt >>> I stumbled upon a reference from James Cameron stating that Skynet has felt guilty for 30 years with regards to the near extinction of human life. He even suggests that Skynet has used time travel in order to setup events so that humanity would win. He even compares John Connor to a Messiah type figure, created intentionally by the 

In [0]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=5 --temperature=1



2019-10-12 12:27:42.009571: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 12:27:42.009843: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2683800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 12:27:42.009884: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.

Model prompt >>> Hi, my name is Mark Farragher. I help C# developers learn computer vision and machine learning in 6 weeks without requiring complex math or Python.
======================================== SAMPLE 1 ========================================


I helped the Java community learn how to work on Windows on the CLR and have fun with it and make it better.

I have created various language related apps and written up 700 lines in a co

In [0]:
#!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=5 --temperature=1

!python3 src/interactive_conditional_samples.py --model_name='345M'  --top_k 40 --temperature 0.9 



2019-10-12 13:59:20.214036: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-10-12 13:59:20.214305: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x294b800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 13:59:20.214343: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version






Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

Model prompt >>> (137378602389) Ioannis Agathocleous: hi
======================================== SAMPLE 1 ========================================
! my name is Ioannis Agathocleous. My family name is Agathocleous. my birthday is March 25. I'm 26 and I'm from Greece. I am studying philosophy at university. I've been married for about 4 years, but ha